In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 【設定】 train.csvのファイルパス（適宜書き換えること）
FILE_PATH = "../AI-Olympic_Mari_Study_Log/joai-competition-2026/train.csv"


def inspect_data():
    print("Loading data... (This might take a while)")
    # 全データを読むと重い場合があるので、最初の10万行だけ読む手もあるが、一旦全部読む
    df = pd.read_csv(FILE_PATH)

    print("\nData Overview:")
    print(df.info())
    print("\nFirst 5 rows:")
    print(df.head())

    # 脳活動のカラムを特定（'AUD'などで始まるカラムと仮定）
    # Overviewによると、AUDp_l, AUDs_l ... VISrl_r といった名前らしい
    # 除外カラム: id, sample_id, mouse_id, day_n, time, lever
    exclude_cols = ["id", "sample_id", "mouse_id", "day_n", "time", "lever"]
    brain_cols = [c for c in df.columns if c not in exclude_cols]

    print(f"\nBrain Activity Columns detected: {len(brain_cols)} columns")
    print(f"Examples: {brain_cols[:5]} ...")

    # --- 可視化: ランダムな1つのsample_idを選んでプロット ---
    unique_samples = df["sample_id"].unique()
    target_sample = unique_samples[0]  # とりあえず最初のサンプルを見る

    sample_data = df[df["sample_id"] == target_sample]

    fig, ax1 = plt.subplots(figsize=(12, 6))

    # 左軸: 脳活動（全部描くと見にくいので、最初の5つだけ描画）
    # もしヒートマップが見たければ、sns.heatmapを使う手もある
    for col in brain_cols[:5]:
        ax1.plot(
            sample_data["time"], sample_data[col], alpha=0.5, label=f"Brain: {col}"
        )

    ax1.set_xlabel("Time")
    ax1.set_ylabel("Brain Activity (Fluorescence)", color="blue")
    ax1.tick_params(axis="y", labelcolor="blue")

    # 右軸: レバー位置 (Target)
    ax2 = ax1.twinx()
    ax2.plot(
        sample_data["time"],
        sample_data["lever"],
        color="red",
        linewidth=2,
        linestyle="--",
        label="Target: Lever",
    )
    ax2.set_ylabel("Lever Position", color="red")
    ax2.tick_params(axis="y", labelcolor="red")

    plt.title(f"Brain Activity vs Lever Position (Sample ID: {target_sample})")
    fig.tight_layout()
    plt.show()

    # --- 相関行列 (Heatmap) ---
    # レバーと各脳領域の相関を見る
    print("\nCalculating correlation with Lever...")
    correlations = (
        df[brain_cols + ["lever"]].corr()["lever"].sort_values(ascending=False)
    )

    plt.figure(figsize=(10, 8))
    # レバーと相関が高い上位10個と下位10個を表示
    top_corr = correlations.head(11)  # lever自身含む
    sns.barplot(x=top_corr.values, y=top_corr.index)
    plt.title("Top Brain Regions correlated with Lever")
    plt.show()


if __name__ == "__main__":
    inspect_data()

In [ ]:
# =========================================================
# 🛠️ Operation "Watchtower" - Validation & Early Stopping
# =========================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  # ★これを使う許可は出ている

# ---------------------------------------------------------
# 1. データ読み込み & ラグ特徴量 (前回と同じ)
# ---------------------------------------------------------
train_path = "../AI-Olympic_Mari_Study_Log/joai-competition-2026/train.csv"
test_path = "../AI-Olympic_Mari_Study_Log/joai-competition-2026/test.csv"
sub_path = "../AI-Olympic_Mari_Study_Log/joai-competition-2026/sample_submission.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sample_sub = pd.read_csv(sub_path)


# ラグ特徴量作成 (Lag=5)
def add_lag_features(df, lag_counts=5):
    df_lagged = df.copy()
    exclude_cols = ["id", "sample_id", "mouse_id", "day_n", "time", "lever"]
    brain_cols = [c for c in df.columns if c not in exclude_cols]

    for i in range(1, lag_counts + 1):
        shifted = df[brain_cols].shift(i)
        shifted.columns = [f"{c}_lag{i}" for c in brain_cols]
        df_lagged = pd.concat([df_lagged, shifted], axis=1)
    return df_lagged.fillna(0)


print(f"Original Train Shape: {X_train.shape}")

# ★重要★ データを「学習用(80%)」と「検証用(20%)」に分割
# shuffle=True でランダムに混ぜる
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print(f"Training set:   {X_tr.shape}")
print(f"Validation set: {X_val.shape}  <-- This is the Truth.")


# ---------------------------------------------------------
# 2. モデル定義 (基本は同じだが、少し改良)
# ---------------------------------------------------------
class MomentumMLP:
    def __init__(
        self, input_size, hidden_size, output_size, learning_rate=0.01, momentum=0.9
    ):
        self.params = {}
        # He Init
        self.params["W1"] = np.random.randn(input_size, hidden_size) * np.sqrt(
            2.0 / input_size
        )
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = np.random.randn(hidden_size, output_size) * np.sqrt(
            2.0 / hidden_size
        )
        self.params["b2"] = np.zeros(output_size)

        self.v = {}
        for key in self.params.keys():
            self.v[key] = np.zeros_like(self.params[key])

        self.lr = learning_rate
        self.momentum = momentum

    def predict(self, x):
        a1 = np.dot(x, self.params["W1"]) + self.params["b1"]
        z1 = np.maximum(0, a1)
        a2 = np.dot(z1, self.params["W2"]) + self.params["b2"]
        return a2

    def loss(self, x, t):
        y = self.predict(x)
        return 0.5 * np.mean((y - t) ** 2)

    # gradient, updateは前回と同じなので省略（そのまま使うこと）
    def gradient(self, x, t):
        grads = {}
        batch_num = x.shape[0]
        W1, W2 = self.params["W1"], self.params["W2"]
        a1 = np.dot(x, W1) + self.params["b1"]
        z1 = np.maximum(0, a1)
        y = np.dot(z1, W2) + self.params["b2"]

        dy = (y - t) / batch_num
        grads["W2"] = np.dot(z1.T, dy)
        grads["b2"] = np.sum(dy, axis=0)
        dz1 = np.dot(dy, W2.T)
        da1 = dz1 * (a1 > 0)
        grads["W1"] = np.dot(x.T, da1)
        grads["b1"] = np.sum(da1, axis=0)
        return grads

    def update(self, x, t):
        grads = self.gradient(x, t)
        for key in self.params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            self.params[key] += self.v[key]


# ---------------------------------------------------------
# 3. 学習実行 (監視付き)
# ---------------------------------------------------------
input_size = X_tr.shape[1]
hidden_size = 512  # 少し減らす（1024は過学習しすぎかも）
output_size = 1
learning_rate = 0.005
momentum = 0.9

model = MomentumMLP(input_size, hidden_size, output_size, learning_rate, momentum)

iters_num = 5000
batch_size = 512
train_size = X_tr.shape[0]

train_loss_list = []
val_loss_list = []  # ★検証用スコアの記録

print(f"Start Training (Hidden={hidden_size})... Watch Validation Loss!")

for i in range(iters_num):
    # 学習用データからミニバッチ
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = X_tr[batch_mask]
    t_batch = y_tr[batch_mask]

    model.update(x_batch, t_batch)

    # 記録 (200回に1回)
    if i % 200 == 0:
        # Train Loss (今のミニバッチ)
        loss_tr = model.loss(x_batch, t_batch)

        # ★Validation Loss (検証データ全部で測る)
        # ※重い場合は一部サンプリングしてもいいが、今回はRTX3070の力を信じて全部やる
        loss_val = model.loss(X_val, y_val)

        train_loss_list.append(loss_tr)
        val_loss_list.append(loss_val)

        print(f"Iter {i} | Train: {loss_tr:.4f} | Val: {loss_val:.4f}")

    # Decay
    if i > 0 and i % 1500 == 0:
        model.lr *= 0.5

# ---------------------------------------------------------
# 4. 可視化 (真実のグラフ)
# ---------------------------------------------------------
plt.plot(train_loss_list, label="Train Loss (Memory)")
plt.plot(
    val_loss_list, label="Val Loss (Reality)", linewidth=2, color="red"
)  # ★これを見ろ！
plt.title("Train vs Validation Loss")
plt.xlabel("Iterations (x200)")
plt.ylabel("MSE")
plt.legend()
plt.show()

# 予測と提出
# 本当は「Val Lossが一番低かった時のモデル」を使うのがベストだが、
# まずは現状を確認してからだ。
y_pred = model.predict(X_test)
sample_sub["lever"] = y_pred
sample_sub.to_csv("submission_watchtower.csv", index=False)

Original Train Shape: (223832, 264)
Training set:   (179065, 264)
Validation set: (44767, 264)  <-- This is the Truth.
Start Training (Hidden=512)... Watch Validation Loss!
Iter 0 | Train: 1.0185 | Val: 1.0690
Iter 200 | Train: 0.7144 | Val: 0.6517
Iter 400 | Train: 0.5804 | Val: 0.6366
Iter 600 | Train: 0.5809 | Val: 0.5914
Iter 800 | Train: 0.5427 | Val: 0.5926
Iter 1000 | Train: 0.5283 | Val: 0.5847
Iter 1200 | Train: 0.6818 | Val: 0.5927
Iter 1400 | Train: 0.5666 | Val: 0.5715
Iter 1600 | Train: 0.4999 | Val: 0.5610
Iter 1800 | Train: 0.5563 | Val: 0.5615
Iter 2000 | Train: 0.5541 | Val: 0.5934


In [ ]:
pip install scikit-learn